In [1]:
import pandas as pd
import numpy as np
import ipaddress as ip
from urllib.parse import urlparse
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import whois
import sklearn.ensemble as ek
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import tldextract

df = pd.read_csv("data.csv")
df = df.sample(frac=1).reset_index(drop=True)
df.head()

E:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
E:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,url,label
0,en.wikipedia.org/wiki/Category:Eyewear_retaile...,good
1,answers.com/topic/golden-state-warriors,good
2,youtube.com/watch?v=ClEoWXyaSbI,good
3,math.mcgill.ca/,good
4,ovguide.com/pals-of-the-golden-west-9202a8c040...,good


In [2]:
# Method to count number of dots
def countdots(url):  
    url = str(url)
    return url.count('.')

# Method to count number of delimeters
def countdelim(url):
    url = str(url)
    count = 0
    delim=[';','_','?','=','&']
    for each in url:
        if each in delim:
            count = count + 1
    
    return count

# Is IP addr present as th hostname, let's validate

import ipaddress as ip #works only in python 3

def is_ip(url):
    try:
        if ip.ip_address(url):
            return 1
    except:
        return 0


def isPresentDSlash(url):
    url = str(url)
    return url.count('//')

def countSubDir(url):
    url = str(url)
    return url.count('/')

def countSubDomain(subdomain):
    if not subdomain:
        return 0
    else:
        return len(subdomain.split('.'))
    
def getFeatures(url, label):
    result = []
    url = str(url)
    result.append(url)
    ext = tldextract.extract(url)
    
    result.append(countdots(url))
    result.append(countdelim(url))
    result.append(is_ip(url))
    result.append(isPresentDSlash(url))
    result.append(countSubDir(url))
    result.append(countSubDomain(ext.subdomain))
    result.append(len(url))
    
    result.append(label)
    return result
    
    
    
def change_lable(x):
    if x == 'bad':
        return 0
    elif x == 'good':
        return 1
    else:
        print("type error")

In [3]:
df['label'] = df['label'].map(lambda x: change_lable(x))
df.head()

,url,label
0,en.wikipedia.org/wiki/Category:Eyewear_retaile...,1
1,answers.com/topic/golden-state-warriors,1
2,youtube.com/watch?v=ClEoWXyaSbI,1
3,math.mcgill.ca/,1
4,ovguide.com/pals-of-the-golden-west-9202a8c040...,1


In [4]:
df['No of dots'] = df['url'].map(lambda x: countdots(x))
df['No of delim'] = df['url'].map(lambda x: countdelim(x))
df['Is IP'] = df['url'].map(lambda x: is_ip(x))
df['No of DSlash'] = df['url'].map(lambda x: isPresentDSlash(x))
df['No of SubDir'] = df['url'].map(lambda x: countSubDir(x))
df['No of SubDomain'] = df['url'].map(lambda x: countSubDomain(x))
df['Len of url'] = df['url'].map(lambda x: len(x))
df['url'] = df['url'].map(lambda x: ('http://' + x))
df.head()

,url,label,No of dots,No of delim,Is IP,No of DSlash,No of SubDir,No of SubDomain,Len of url
0,http://en.wikipedia.org/wiki/Category:Eyewear_...,1,2,5,0,0,2,3,70
1,http://answers.com/topic/golden-state-warriors,1,1,0,0,0,2,2,39
2,http://youtube.com/watch?v=ClEoWXyaSbI,1,1,2,0,0,1,2,31
3,http://math.mcgill.ca/,1,2,0,0,0,1,3,15
4,http://ovguide.com/pals-of-the-golden-west-920...,1,1,0,0,0,1,2,68


In [5]:
from sklearn import cross_validation
X = df.drop(['url', 'label'], axis = 1).values
y = df['label'].values
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y ,test_size=0.2)

In [6]:
#Decision tree
from sklearn import tree
from sklearn.metrics import confusion_matrix
dt = tree.DecisionTreeClassifier(max_depth = 10)
dt.fit(X_train, y_train)
r = dt.predict(X)
matrix = confusion_matrix(y, r)
TP = matrix[0][0]
FP = matrix[0][1]
FN = matrix[1][0]
TN = matrix[1][1]
Acc = (TP + TN) / (TP + FP + TN + FN)
Rec = TP / (TP + FN)
print("Accuracy: %.2f  Recall: %.2f" % (Acc, Rec))

Accuracy: 0.85  Recall: 0.81


In [7]:
#输入单个恶意url进行测试。假冒工商银行网站
URL = 'http://www.1cbc.com.cn/icbc/'
result = pd.DataFrame(columns = ('url', 'label', 'No of dots', 'No of delim', 'Is IP', 'No of DSlash', 'No of SubDir', 'No of SubDomain', 'Len of url'))
results = getFeatures(URL, 1)
result.loc[0] = results
result = result.drop(['url','label'],axis=1).values
print(dt.predict(result))

[0]


In [8]:
#Random Forest
import sklearn.ensemble as ek
clf = ek.RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
r = clf.predict(X)
matrix = confusion_matrix(y, r)
TP = matrix[0][0]
FP = matrix[0][1]
FN = matrix[1][0]
TN = matrix[1][1]
Acc = (TP + TN) / (TP + FP + TN + FN)
Rec = TP / (TP + FN)
print("Accuracy: %.2f  Recall: %.2f" % (Acc, Rec))

Accuracy: 0.86  Recall: 0.81


In [9]:
#输入单个恶意url进行测试。假冒工商银行网站
URL = 'http://www.1cbc.com.cn/icbc/'
result = pd.DataFrame(columns = ('url', 'label', 'No of dots', 'No of delim', 'Is IP', 'No of DSlash', 'No of SubDir', 'No of SubDomain', 'Len of url'))
results = getFeatures(URL, 1)
result.loc[0] = results
result = result.drop(['url','label'],axis=1).values
print(clf.predict(result))

[0]
